In [1]:
import os
import pandas as pd
import datetime as dt
import shutil

In [2]:
# получим директорию папки "Загрузки" в системе
from pathlib import Path
downloads_path = str(Path.home() / "Downloads")

In [3]:
# конечная папка для выгрузки ЕМИАС данных
# вставляем между кавычек путь к этой папке на своем компьютере
data_emias_xlsx_path = r'\\t999\Сетевой диск\pdn\data\emias_data\xlsx'

In [4]:
data_emias_xlsx_path = data_emias_xlsx_path.split('\\')
data_emias_xlsx_path = os.path.join(r'\\', *data_emias_xlsx_path)

In [5]:
today = dt.datetime.today().date()

for file in os.listdir(downloads_path):
    if '.xlsx' in file:
        initial_file_path = os.path.join(downloads_path, file)
        if 'DAT' in  file and today.strftime('%Y-%m-%d') in file:
            final_file_path = os.path.join(data_emias_xlsx_path, f'{file[:58]}.xlsx')
            shutil.move(initial_file_path, final_file_path)

In [6]:
today.strftime('%Y-%m-%d')

'2022-08-15'

In [7]:
import sys
path_to_pdn = r"C:\Users\murad_satabaev\Documents\GitHub\pdn_" # сюда внутри кавычек вставляем путь, где ледит репозиторий пдн
sys.path.insert(0, path_to_pdn)

In [8]:
import os
import zipfile
import datetime
import numpy as np
import pandas as pd
from shutil import make_archive
from pdn.utils.utils import get_filenames
from pdn.configs.paths import EMIAS_SUBFOLDER_LOADS

In [9]:
# cловарь "номер_dat": "месяц", в который мы добавляем список актуальных месяцев
dat_month = dict()
# dat_month['may'] = '3196'
# dat_month['june'] = '3437'
dat_month['july'] = '3582'
dat_month['august'] = '3833'

In [10]:
# задаем дату, за которую нужно проверить файлы, и ставим ее в нужный формат строки
# date_ = datetime.date(2022, 7, 14)
date_ = datetime.datetime.today().date()
date_ymd = date_.strftime('%Y-%m-%d')
date_y_m_d = date_.strftime('%Y_%m_%d')
print(date_ymd)

2022-08-15


In [11]:
# конечно, было бы хорошо заменить ввод даты на автоматическое считывание сегодняшней даты, но это будет реализовано после 
# того, как система в общем придет в динамическое спокойствие и весь код будет прописан
datetime.datetime.now().date()
# этот кусок кода вставить в переменную date_
# пока он не нужен, не обращать на него внимания

datetime.date(2022, 8, 15)

In [12]:
# получаем список файлов в папке EMIAS_SUBFOLDER_LOADS, в названии которых есть date_ymd
file_list = get_filenames(EMIAS_SUBFOLDER_LOADS, date_ymd)
print(*file_list)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx\DAT_3582_выгрузка_списков_на_обзвон_помощниками_2022-08-15.xlsx \\t999\Сетевой диск\pdn\data\emias_data\xlsx\DAT_3833_выгрузка_списков_на_обзвон_помощниками_2022-08-15.xlsx


In [13]:
# зададим путь папки для сохранения папок с файлами - для этого на место пути тут просто скопируйте свой путь из адроесной
# строки и вставьте
path = r'C:\Users\murad_satabaev\Documents\GitHub\final_framework\Утренний отбор пациентов без прикрепления к врачу'
splitted_root_path = path.split('\\')
root_path = os.path.join(splitted_root_path[0], os.sep, *splitted_root_path[1:])

In [14]:
# если в системе нет папки с нужной датой, то данная ячейка создает ее и добавляет ее к пути
export_path = os.path.join(root_path, date_.strftime('%Y%m%d'))

if not os.path.isdir(export_path):
    os.mkdir(os.path.join(root_path, date_.strftime('%Y%m%d')))

In [15]:
# зададим список поликлиник, по которым нужно проверять наличие пациентов без прикрепления
# mo_to_check_set = set()
# mo_to_check_list = ['ГБУЗ "ГП № 134 ДЗМ"', 'ГБУЗ "ГП № 218 ДЗМ"', 'ГБУЗ "ГП № 22 ДЗМ"', 'ГБУЗ ГП № 191 ДЗМ', 
#                    'ГБУЗ ГП № 220 ДЗМ','ГБУЗ ДКЦ 1 ДЗМ', 'КДЦ 4']
# for mo in mo_to_check_list: 
#     mo_to_check_set.add(mo.replace('ГБУЗ', '').replace('№', '').replace('ДЗМ', '').replace('"', '').replace('"', '') \
#     .strip().replace('  ', ' ').replace(' ', '_'))
# mo_to_check_set.add(np.NaN)
mo_to_check_set = {'ГП_64', 'ГП_191', 'ДКЦ_1', 'КДЦ_4', 'ДЦ_5', 'ГП_2', 'ГП_210', 'ГП_107', 'ГП_23', 'ГП_66', 'ДЦ_3', 'ГП_170', 
           'ГП_22', 'КДП_121', 'КДЦ_6', 'ГП_214', 'ГП_218', 'ГП_109', 'ГП_134', 'ГП_36', 'ГП_6', 'ГП_220', 'ГП_11'} 

In [16]:
export_path

'C:\\Users\\murad_satabaev\\Documents\\GitHub\\final_framework\\Утренний отбор пациентов без прикрепления к врачу\\20220815'

In [17]:
len(mo_to_check_set)

23

In [18]:
%%time
# последний этап - формирование файлов по месяцамам и МО
for file_path in file_list:
        for month, dat  in dat_month.items():
            if dat in file_path:                
                
                df = pd.read_excel(file_path)
                df = df.rename(columns={'Умер': 'dead'})
                df[['emiasid', 'Pol_number']] = df[['emiasid', 'Pol_number']].astype(str)
                df = df[(df.dead == 'нет') & (df.doctor_fio.isna())]             
                
                mo_set = set(df.mo.unique())
                mo_dict = dict()
                for mo in mo_set:
                    if not pd.isna(mo):
                        mo_dict[mo] = mo.replace('ГБУЗ', '').replace('№', '').replace('ДЗМ', '').replace('"', '') \
                        .replace('"', '').strip().replace('  ', ' ').replace(' ', '_')
                        print(mo_dict[mo])
                    else:
                        mo_dict[mo] = mo               
                for key, value in mo_dict.items():
                    if value in mo_to_check_set:
                        if pd.isna(value) and set(df.mo.isna()):
                            df[df.mo.isna()].to_excel(os.path.join(export_path, f"Пустое_mo_{month}_пустые_doctor_fio.xlsx"),
                                index=False)
                        else:
                            df[df.mo == key].to_excel(os.path.join(export_path, f"{value}_{month}_пустые_doctor_fio.xlsx"),
                                index=False)

ГП_191
ДКЦ_1
ГП_210
ДКЦ_1
ГП_107
ГП_9
ГП_11
КДЦ_4
CPU times: total: 1min 3s
Wall time: 1min 4s


In [19]:
########################################################################################################################

In [20]:
########################################################################################################################

In [21]:
folder_list = [file for file in os.listdir(root_path) if os.path.isdir(file) and '.' not in file]
folder_list.sort()


In [22]:
try:
    folder_day_before = folder_list[-2]
    day_before_files = set(os.listdir(os.path.join(root_path, folder_day_before)))
    day_before_files
except IndexError:
    exit()

In [23]:
day_before_files

{'ГП_107_august_пустые_doctor_fio.xlsx',
 'ГП_210_august_пустые_doctor_fio.xlsx',
 'ГП_218_august_пустые_doctor_fio.xlsx',
 'ГП_220_august_пустые_doctor_fio.xlsx',
 'ГП_220_july_пустые_doctor_fio.xlsx',
 'ГП_22_august_пустые_doctor_fio.xlsx',
 'ГП_22_july_пустые_doctor_fio.xlsx',
 'ГП_66_august_пустые_doctor_fio.xlsx',
 'КДЦ_4_august_пустые_doctor_fio.xlsx',
 'КДЦ_4_july_пустые_doctor_fio.xlsx',
 'КДЦ_6_august_пустые_doctor_fio.xlsx'}

In [24]:
folder_today = folder_list[-1]
today_files = set(os.listdir(os.path.join(root_path, folder_today)))
today_files

{'ГП_107_august_пустые_doctor_fio.xlsx',
 'ГП_11_august_пустые_doctor_fio.xlsx',
 'ГП_191_july_пустые_doctor_fio.xlsx',
 'ГП_210_august_пустые_doctor_fio.xlsx',
 'ДКЦ_1_august_пустые_doctor_fio.xlsx',
 'ДКЦ_1_july_пустые_doctor_fio.xlsx',
 'КДЦ_4_august_пустые_doctor_fio.xlsx'}

In [25]:
common_files = day_before_files & today_files
common_files

{'ГП_107_august_пустые_doctor_fio.xlsx',
 'ГП_210_august_пустые_doctor_fio.xlsx',
 'КДЦ_4_august_пустые_doctor_fio.xlsx'}

In [26]:
for file_name in common_files:
    day_before = pd.read_excel(os.path.join(root_path, folder_day_before, file_name))
    day_before.emiasid = day_before.emiasid.astype(str).str.replace('\.0', '')
    today = pd.read_excel(os.path.join(root_path, folder_today, file_name))
    today.emiasid = today.emiasid.astype(str).str.replace('\.0', '')
    result = today[today.emiasid.isin(day_before.emiasid)]
    result[['emiasid', 'Pol_number']] = result[['emiasid', 'Pol_number']].astype(str)
    if result.shape[0] > 0:
        result.to_excel(os.path.join(root_path, f'{file_name[:-5]}_{date_y_m_d}{file_name[-5:]}'), index=False)

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_3756\1581390915.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  day_before.emiasid = day_before.emiasid.astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_3756\1581390915.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  today.emiasid = today.emiasid.astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_3756\1581390915.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[['emiasid', 'Pol_number']] = result[['emiasid', 'Pol_number']].astype(str)
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_3756\1581390915.py

In [27]:
# exit()